### This notebook was used to convert the original PhaseNet weights to SeisBench and ensure binary compatibility. It is not intended for further use, but only serves as a reference

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/munchmej/code/PhaseNet/')

In [3]:
import tensorflow as tf
from keras.models import load_model
import os
from model import Model
from run import set_config
from data_reader import Config, DataReader, DataReader_test, DataReader_pred, DataReader_mseed
import argparse

Using TensorFlow backend.


In [5]:
saver = tf.compat.v1.train.import_meta_graph('/home/munchmej/code/PhaseNet/model/190703-214543/model_95.ckpt.meta')

In [11]:
with tf.compat.v1.Session() as sess:
    saver.restore(sess, tf.compat.v1.train.latest_checkpoint('/home/munchmej/code/PhaseNet/model/190703-214543/'))
    graph = sess.graph

In [13]:
graph.get_operations()

[<tf.Operation 'create_inputs/Placeholder' type=Placeholder>,
 <tf.Operation 'create_inputs/Placeholder_1' type=Placeholder>,
 <tf.Operation 'create_inputs/padding_fifo_queue' type=PaddingFIFOQueueV2>,
 <tf.Operation 'create_inputs/padding_fifo_queue_enqueue' type=QueueEnqueueV2>,
 <tf.Operation 'create_inputs/Placeholder_2' type=Placeholder>,
 <tf.Operation 'create_inputs/Placeholder_3' type=Placeholder>,
 <tf.Operation 'create_inputs/padding_fifo_queue_1' type=PaddingFIFOQueueV2>,
 <tf.Operation 'create_inputs/padding_fifo_queue_1_enqueue' type=QueueEnqueueV2>,
 <tf.Operation 'Input_Batch/padding_fifo_queue_DequeueMany/n' type=Const>,
 <tf.Operation 'Input_Batch/padding_fifo_queue_DequeueMany' type=QueueDequeueManyV2>,
 <tf.Operation 'Input_Batch/padding_fifo_queue_1_DequeueMany/n' type=Const>,
 <tf.Operation 'Input_Batch/padding_fifo_queue_1_DequeueMany' type=QueueDequeueManyV2>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'global_step/initial_value' type=Const>,
 <tf.Operati

In [4]:
parser = argparse.ArgumentParser()

parser.add_argument("--mode",
                  default="train",
                  help="train/valid/test/debug")

parser.add_argument("--epochs",
                  default=100,
                  type=int,
                  help="number of epochs (default: 10)")

parser.add_argument("--batch_size",
                  default=200,
                  type=int,
                  help="batch size")

parser.add_argument("--learning_rate",
                  default=0.01,
                  type=float,
                  help="learning rate")

parser.add_argument("--decay_step",
                  default=-1,
                  type=int,
                  help="decay step")

parser.add_argument("--decay_rate",
                  default=0.9,
                  type=float,
                  help="decay rate")

parser.add_argument("--momentum",
                  default=0.9,
                  type=float,
                  help="momentum")

parser.add_argument("--filters_root",
                  default=8,
                  type=int,
                  help="filters root")

parser.add_argument("--depth",
                  default=5,
                  type=int,
                  help="depth")

parser.add_argument("--kernel_size",
                  nargs="+",
                  type=int,
                  default=[7, 1],
                  help="kernel size")

parser.add_argument("--pool_size",
                  nargs="+",
                  type=int,
                  default=[4, 1],
                  help="pool size")

parser.add_argument("--drop_rate",
                  default=0,
                  type=float,
                  help="drop out rate")

parser.add_argument("--dilation_rate",
                  nargs="+",
                  type=int,
                  default=[1, 1],
                  help="dilation rate")

parser.add_argument("--loss_type",
                  default="cross_entropy",
                  help="loss type: cross_entropy, IOU, mean_squared")

parser.add_argument("--weight_decay",
                  default=0,
                  type=float,
                  help="weight decay")

parser.add_argument("--optimizer",
                  default="adam",
                  help="optimizer: adam, momentum")

parser.add_argument("--summary",
                  default=True,
                  type=bool,
                  help="summary")

parser.add_argument("--class_weights",
                  nargs="+",
                  default=[1, 1, 1],
                  type=float,
                  help="class weights")

parser.add_argument("--log_dir",
                  default="log",
                  help="Tensorboard log directory (default: log)")

parser.add_argument("--model_dir",
                  default=None,
                  help="Checkpoint directory (default: None)")

parser.add_argument("--num_plots",
                  default=10,
                  type=int,
                  help="Plotting trainning results")

parser.add_argument("--tp_prob",
                  default=0.3,
                  type=float,
                  help="Probability threshold for P pick")

parser.add_argument("--ts_prob",
                  default=0.3,
                  type=float,
                  help="Probability threshold for S pick")

parser.add_argument("--input_length",
                  default=None,
                  type=int,
                  help="input length")

parser.add_argument("--input_mseed",
                  action="store_true",
                  help="mseed format")

parser.add_argument("--data_dir",
                  default="./dataset/waveform_pred/",
                  help="Input file directory")

parser.add_argument("--data_list",
                  default="./dataset/waveform.csv",
                  help="Input csv file")

parser.add_argument("--train_dir",
                  default="./dataset/waveform_train/",
                  help="Input file directory")

parser.add_argument("--train_list",
                  default="./dataset/waveform.csv",
                  help="Input csv file")

parser.add_argument("--valid_dir",
                  default=None,
                  help="Input file directory")

parser.add_argument("--valid_list",
                  default=None,
                  help="Input csv file")

parser.add_argument("--output_dir",
                  default=None,
                  help="Output directory")

parser.add_argument("--plot_figure",
                  action="store_true",
                  help="If plot figure for test")

parser.add_argument("--save_result",
                  action="store_true",
                  help="If save result for test")

parser.add_argument("--fpred",
                  default="picks",
                  help="Ouput filename for test")

parser.add_argument("-f",
                  default="picks",
                  help="Ouput filename for test")

args = parser.parse_args()

In [5]:
coord = tf.train.Coordinator()
data_reader = DataReader_mseed("../../PhaseNet/demo/mseed/", "../../PhaseNet/demo/fname.csv", 1, coord)
config = set_config(args, data_reader)

with tf.compat.v1.name_scope('Input_Batch'):
    batch = data_reader.dequeue(args.batch_size)
    
model = Model(config, batch, "pred")

In [6]:
sess_config = tf.compat.v1.ConfigProto()
sess_config.gpu_options.allow_growth = True
sess_config.log_device_placement = False

In [7]:
with tf.compat.v1.Session(config=sess_config) as sess:
    threads = data_reader.start_threads(sess, n_threads=8)

    # saver = tf.compat.v1.train.Saver(tf.compat.v1.global_variables(), max_to_keep=5)
    # init = tf.compat.v1.global_variables_initializer()
    # sess.run(init)

    # latest_check_point = tf.train.latest_checkpoint("/home/munchmej/code/PhaseNet/model/190703-214543/")
    # saver.restore(sess, latest_check_point)

../../PhaseNet/demo/mseed/CCC.mseed:   0%|          | 0/1680 [00:00<?, ?it/s]
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/munchmej/anaconda3/envs/eqtransformer/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/munchmej/anaconda3/envs/eqtransformer/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/munchmej/code/PhaseNet/data_reader.py", line 419, in thread_main
    self.fname_placeholder: f"{fname}_{i*self.input_length}"})
  File "/home/munchmej/anaconda3/envs/eqtransformer/lib/python3.7/site-packages/tensorflow_core/python/client/session.py", line 956, in run
    run_metadata_ptr)
  File "/home/munchmej/anaconda3/envs/eqtransformer/lib/python3.7/site-packages/tensorflow_core/python/client/session.py", line 1103, in _run
    raise RuntimeError('Attempted to use a closed Session.')
RuntimeError: Attempted to use a closed Session.

